In [19]:
import pybithumb 
import datetime 
import numpy as np 
import pandas as pd 
import time 
import os 
import math 
import random

# 데이터 준비

In [20]:
def get_target_price(ticker): 
    df = pybithumb.get_ohlcv(ticker) 
    yesterday = df.iloc[-2] 
    today_open = yesterday['close'] 
    yesterday_high = yesterday['high'] 
    yesterday_low = yesterday['low'] 
    target = today_open + (yesterday_high - yesterday_low) * 0.5 
    return target 

In [21]:
df = pybithumb.get_ohlcv("BTC") 
df['range'] = (df['high'] - df['low']) * 0.5 
df['target'] = df['open'] + df['range'].shift(1) 
df.to_csv('btc.csv',index=False)

In [22]:
df

,open,high,low,close,volume,range,target
time,,,,,,,
2013-12-27 00:00:00,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0,NaN
2013-12-28 00:00:00,750000.0,750000.0,750000.0,750000.0,12.000000,0.0,759000.0
2013-12-29 00:00:00,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0,750000.0
2013-12-30 00:00:00,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0,751000.0
2013-12-31 00:00:00,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0,784000.0
...,...,...,...,...,...,...,...
2021-05-24 00:00:00,42220000.0,47180000.0,38536000.0,45312000.0,9187.329718,4322000.0,45793500.0
2021-05-25 00:00:00,45320000.0,49074000.0,44640000.0,46080000.0,7426.003128,2217000.0,49642000.0
2021-05-26 00:00:00,46147000.0,48888000.0,45431000.0,47741000.0,4310.370082,1728500.0,48364000.0


변동성 돌파 전략에서는 각 거래의 고가가 목표가 이상이면 수행된 것으로 생각하면 된다. 

In [23]:
df['ror'] = np.where(df['high'] > df['target'], df['close'] / df['target'], 1) 


In [24]:
df

,open,high,low,close,volume,range,target,ror
time,,,,,,,,
2013-12-27 00:00:00,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0,NaN,1.000000
2013-12-28 00:00:00,750000.0,750000.0,750000.0,750000.0,12.000000,0.0,759000.0,1.000000
2013-12-29 00:00:00,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0,750000.0,1.000000
2013-12-30 00:00:00,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0,751000.0,1.022636
2013-12-31 00:00:00,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0,784000.0,0.979592
...,...,...,...,...,...,...,...,...
2021-05-24 00:00:00,42220000.0,47180000.0,38536000.0,45312000.0,9187.329718,4322000.0,45793500.0,0.989485
2021-05-25 00:00:00,45320000.0,49074000.0,44640000.0,46080000.0,7426.003128,2217000.0,49642000.0,1.000000
2021-05-26 00:00:00,46147000.0,48888000.0,45431000.0,47741000.0,4310.370082,1728500.0,48364000.0,0.987119


In [25]:
ror = df['ror'].cumprod()[-2] 
print(ror)

53.733478118287955


In [98]:
df = pybithumb.get_ohlcv("BTC") 
df = df['2018']
df['range'] = (df['high'] - df['low']) * 0.5 
df['target'] = df['open'] + df['range'].shift(1) 
df['ror'] = np.where(df['high'] > df['target'], df['close'] / df['target'], 1) 
ror = df['ror'].cumprod()[-2] 
print(ror)

1.1817568351455974


# 수수료에서 오는 수익률 차이 계산하기 

In [90]:
# 초기 자본금 5000만원, 매수가 4000만원, 매도가 4800만원 

투자원금 = 50000000
매수체결가 = 40000000
매도체결가 = 48000000

In [91]:
수익률 = 매도체결가/매수체결가
수익률

1.2

In [92]:
수익률_수수료 = 매도체결가/매수체결가 * (1-0.15/100) * (1-0.15/100) 
수익률_수수료

1.1964027

In [93]:
수익률 - 수익률_수수료

0.0035973000000000255

In [94]:
df = pybithumb.get_ohlcv("BTC") 
df['range'] = (df['high'] - df['low']) * 0.5 
df['target'] = df['open'] + df['range'].shift(1) 
df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'] * (1-0.15/100) * (1-0.15/100), 1)  
df.to_csv('btc.csv',index=False)


ror = df['ror'].cumprod()[-2] 
ror

2.124662172695581

In [95]:
# 0.003 정도만 빼줘도 얼추 비슷하다. 
df = pybithumb.get_ohlcv("BTC") 
df['range'] = (df['high'] - df['low']) * 0.5 
df['target'] = df['open'] + df['range'].shift(1) 
df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'] - 0.003, 1)  
df.to_csv('btc.csv',index=False)


ror = df['ror'].cumprod()[-2] 
ror

2.141731802327325

In [96]:
df = pybithumb.get_ohlcv("BTC") 
df['range'] = (df['high'] - df['low']) * 0.5 
df['target'] = df['open'] + df['range'].shift(1) 
df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'] - 0.003, 1) 
df['hpr'] = df['ror'].cumprod() 
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100 
print("MDD(%) = {}".format(df['dd'].max()))
df.to_csv('btc.csv',index=False)


MDD(%) = 73.6076379616712


In [97]:
df = pybithumb.get_ohlcv("BTC") 
df['range'] = (df['high'] - df['low']) * 0.5 
df['target'] = df['open'] + df['range'].shift(1) 
df['ror'] = np.where(df['high'] > df['target'], df['close']/df['target'], 1) 
df['hpr'] = df['ror'].cumprod() 
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100 
print("MDD(%) = {}".format(df['dd'].max()))



MDD(%) = 51.34724635380804


# 하이브리드 전략

In [101]:
df = pybithumb.get_ohlcv("BTC") 
df['ma5'] = df['close'].rolling(window=5).mean().shift(1) 
df['range'] = (df['high'] - df['low']) * 0.5 
df['target'] = df['open'] + df['range'].shift(1) 
df['bull'] = df['open'] > df['ma5'] 
df['ror'] = np.where((df['high'] > df['target'])&df['bull'], df['close']/df['target'] * (1-0.15/100) * (1-0.15/100), 1) 
df['hpr'] = df['ror'].cumprod() 
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100 
print("MDD: ", df['dd'].max())
print("ror: ", df['hpr'][-2]) 

MDD:  38.0123833549567
ror:  17.254495270409755


# Hyperparameter optimization

In [105]:
def get_ror(k):
    df = pybithumb.get_ohlcv("BTC") 
    df['ma5'] = df['close'].rolling(window=5).mean().shift(1) 
    df['range'] = (df['high'] - df['low']) * k
    df['target'] = df['open'] + df['range'].shift(1) 
    df['bull'] = df['open'] > df['ma5'] 
    df['ror'] = np.where((df['high'] > df['target'])&df['bull'], df['close']/df['target'] - 0.003, 1) 
    return df['ror'].cumprod()[-2]

for k in np.arange(0.1, 1.0, 0.1): 
    ror = get_ror(k) 
    print(k, ror)
    

0.1 23.55869151935831
0.2 21.13251801914171
0.30000000000000004 16.679247952774997
0.4 20.439266445768123
0.5 17.461749624002817
0.6 9.614981670830101
0.7000000000000001 9.611238031930892
0.8 7.4544785315251065
0.9 4.340238485196269
